In [11]:
import os

In [12]:
%pwd

'c:\\Users\\a12ej56k\\VS code\\Text-Summarizer-Project'

In [3]:

os.chdir('../')

In [20]:
%pwd

'c:\\Users\\a12ej56k\\VS code\\Text-Summarizer-Project'

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path



In [22]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [23]:
class configurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)  
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [24]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} with folllwing info : \n {headers}")
        else:
            logger.info(f"{filename} already exists of size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):

        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:
    config=configurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-01-02 21:23:15,631 : INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-01-02 21:23:15,631 : INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-02 21:23:15,631 : INFO:common:created directory at: artifacts]
[2024-01-02 21:23:15,639 : INFO:common:created directory at: artifacts/data_ingestion]
[2024-01-02 21:23:17,261 : INFO:2127453442:Downloaded artifacts/data_ingestion/data.zip with folllwing info : 
 Server: GitHub.com
Date: Tue, 02 Jan 2024 15:53:34 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"1fd204457edce27029716129e5c560a7"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade
Content-Security-Policy: default-src 'none'; base-uri 'self'; child-src github

BadZipFile: File is not a zip file